# Introduction
[Chainer](http://chainer.org/) とはニューラルネットの実装を簡単にしたフレームワークです。

* 今回は機械翻訳にニューラルネットを適用してみました。

![](./pictures/Chainer.jpg)

* 今回は機械翻訳を行っていただきます。

ハンズオンについての説明
http://qiita.com/GushiSnow/private/b0abf7d3dccafe14fa07

機械翻訳は機械が言語を別の言語に翻訳するものです。

機械翻訳にはいくつか種類があるのでここでも紹介しておきます。

* PBMT(Phrase Base Machine Translation)モデル
 * [moses](http://www.statmt.org/moses/)というオープンソースで使用できるメジャーな機械翻訳のモデルですが、難しすぎて理解できない人を続出させる機械翻訳の鬼門です
* ニューラル機械翻訳
 * 翻訳元単語の辞書ベクトルを潜在空間ベクトルに落とし込み、ニューラルネットで翻訳先言語を学習させる手法

以下では、このChainerを利用しデータを準備するところから実際にNN翻訳モデルを構築し学習・評価を行うまでの手順を解説します。

1. [各種ライブラリ導入](#各種ライブラリ導入) 
2. [機械翻訳のクラス](#機械翻訳のクラス)
3. [翻訳処理を行うforward処理部分](#翻訳処理を行うforward処理部分)
4. [各値を設定](#各値を設定)
5. [実行](#実行)



## 1.各種ライブラリ導入

Chainerの言語処理では多数のライブラリを導入します。



In [1]:
from util.functions import trace
import numpy as np

from chainer import functions, optimizers

from util.chainer_cpu_wrapper import wrapper

from EncoderDecoderModel import EncoderDecoderModel
import subprocess

`導入するライブラリの代表例は下記です。

* `numpy`: 行列計算などの複雑な計算を行なうライブラリ
* `chainer`: Chainerの導入
* `util`:今回の処理で必要なライブラリが入っています。


## 2.機械翻訳のクラス

下記を設定しています。
* ニューラルネットを用いて機械翻訳用のモデルを構成しています。

全体構成

![](./pictures/NN_machine_translation.png)




## 3.翻訳処理を行うforwardに必要なパラメータ設定

下記のコードで必要なパラメータを設定するクラスを定義しています。

In [2]:
class EncoderDecoderModelParameter():
    
    def __init__(self, is_training, src_batch, encoderDecoderModel, trg_batch = None, generation_limit = None):
        self.m = encoderDecoderModel.model
        self.tanh = functions.tanh
        self.lstm = functions.lstm
        self.batch_size = len(src_batch)
        self.src_len = len(src_batch[0])
        self.src_stoi = encoderDecoderModel.src_vocab.stoi
        self.trg_stoi = encoderDecoderModel.trg_vocab.stoi
        self.trg_itos = encoderDecoderModel.trg_vocab.itos
        self.s_c = wrapper.zeros((self.batch_size, encoderDecoderModel.n_hidden))
        self.trg_batch = trg_batch
        self.generation_limit = generation_limit


## 4.翻訳処理を行うEncoder処理部分

下記の論文を参考にしてforward処理を記述しています。

http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf

* Encoder部分
もっとも特徴的な部分は翻訳元言語を逆順にしていることです。そうすることで精度が向上していると述べており、今回の翻訳のNNモデルもそれを再現しています。

この論文でははっきりした要因はわかっていないが、おそらく翻訳先の言語と翻訳元言語の距離が逆順にすることで最初の単語の距離が近くなり、翻訳のタイムラグが少なくなったことが起因していると考えられています。

![](./pictures/encoder.png)



In [3]:
class EncoderDecoderModelEncoding():
    
    def encoding(self, src_batch, parameter, trg_batch = None, generation_limit = None):

#--------Hands on------------------------------------------------------------------#
    # encoding
        #翻訳元言語の末尾</s>を潜在空間に写像し、隠れ層に入力、lstmで出力までをバッチサイズ分行う。
        s_x = wrapper.make_var([parameter.src_stoi('</s>') for _ in range(parameter.batch_size)], dtype=np.int32)
        s_i = parameter.tanh(parameter.m.w_xi(s_x))
        parameter.s_c, s_p = parameter.lstm(parameter.s_c, parameter.m.w_ip(s_i))
        
        #翻訳元言語を逆順に１と同様の処理を行う。   
        for l in reversed(range(parameter.src_len)):
            s_x = wrapper.make_var([parameter.src_stoi(src_batch[k][l]) for k in range(parameter.batch_size)], dtype=np.int32)
            s_i = parameter.tanh(parameter.m.w_xi(s_x))
            parameter.s_c, s_p = parameter.lstm(parameter.s_c, parameter.m.w_ip(s_i) + parameter.m.w_pp(s_p))

        #次のミニバッチ処理のために最終結果をlstmで出力。翻訳の仮説用のリストを保持
        parameter.s_c, s_q = parameter.lstm(parameter.s_c, parameter.m.w_pq(s_p))
        hyp_batch = [[] for _ in range(parameter.batch_size)]
        return s_q, hyp_batch

## 5.翻訳処理を行うDecoder処理部分

* Decoder部分

学習部分と予測部分を実装しています。学習部分ではターゲット先の単語の取得と損失の計算をしています。
またlstmで次回の学習に使用する部分では学習では正解の翻訳、予測では予測した翻訳を使用しています。

![](./pictures/decorder.png)

In [4]:
class EncoderDecoderModelDecoding():
    
    def decoding(self, is_training, src_batch, parameter, s_q, hyp_batch, trg_batch = None, generation_limit = None):

    # decoding
        """
　　     学習と予測をコードしている。
        """
        if is_training:
            #損失の初期化及び答えとなる翻訳先言語の長さを取得。（翻訳元言語と翻訳先言語で長さが異なるため）
            accum_loss = wrapper.zeros(())
            trg_len = len(parameter.trg_batch[0])

            #ニューラルネットの処理は基本的にEncodingと同一であるが、損失計算と翻訳仮説候補の確保の処理が加わっている。
            for l in range(trg_len):
                s_j = parameter.tanh(parameter.m.w_qj(s_q))
                r_y = parameter.m.w_jy(s_j)
                s_t = wrapper.make_var([parameter.trg_stoi(parameter.trg_batch[k][l]) for k in range(parameter.batch_size)], dtype=np.int32)
                #損失計算
                accum_loss += functions.softmax_cross_entropy(r_y, s_t)
                #仮説出力
                output = wrapper.get_data(r_y).argmax(1)

                #仮説確保
                for k in range(parameter.batch_size):
                    hyp_batch[k].append(parameter.trg_itos(output[k]))

                parameter.s_c, s_q = parameter.lstm(parameter.s_c, parameter.m.w_yq(s_t) + parameter.m.w_qq(s_q))
            return hyp_batch, accum_loss
        else:
            """
            予測部分
            １：予測では予測する翻訳言語の長さに制約をしないとニューラル翻訳モデルの性質上、無限に翻訳してしまう可能性があるので、長さに制約を設けている。
            ２：翻訳元言語を逆順に１と同様の処理を行う。
            ３：基本的に学習部分と同一であるが、ミニバッチサイズ分の翻訳仮説の末尾が</s>になったときにDecoding処理が終わるようになっている。
            """
            while len(hyp_batch[0]) < parameter.generation_limit:
                s_j = parameter.tanh(parameter.m.w_qj(s_q))
                r_y = parameter.m.w_jy(s_j)
                output = wrapper.get_data(r_y).argmax(1)

                for k in range(parameter.batch_size):
                    hyp_batch[k].append(parameter.trg_itos(output[k]))

                if all(hyp_batch[k][-1] == '</s>' for k in range(parameter.batch_size)): break

                s_y = wrapper.make_var(output, dtype=np.int32)
                parameter.s_c, s_q = parameter.lstm(parameter.s_c, parameter.m.w_yq(s_y) + parameter.m.w_qq(s_q))

            return hyp_batch
        
#--------Hands on------------------------------------------------------------------#

## 6.翻訳処理を行うforward処理部分

上記の処理を実行するためのメソッドです。

In [5]:
class EncoderDecoderModelForward(EncoderDecoderModel):
    
    def forward(self, is_training, src_batch, trg_batch = None, generation_limit = None):
        parameter = EncoderDecoderModelParameter(is_training, src_batch, self, trg_batch, generation_limit)
        
#--------Hands on------------------------------------------------------------------#
    # encoding
        encoder = EncoderDecoderModelEncoding()
        s_q, hyp_batch = encoder.encoding(src_batch, parameter)
    # decoding
        decoder = EncoderDecoderModelDecoding()
        if is_training:
            return decoder.decoding(is_training, src_batch, parameter, s_q, hyp_batch, trg_batch, generation_limit)
        else:
            return decoder.decoding(is_training, src_batch, parameter, s_q, hyp_batch, trg_batch, generation_limit)
#--------Hands on------------------------------------------------------------------#

## 4.各値を設定

各値を設定

* モードを学習かテストか設定
* 翻訳元言語の設定
* 翻訳先言語の設定
* 語彙の設定
* 潜在空間の設定
* 隠れ層の設定
* 学習回数の設定
* ミニバッチサイズの設定
* 最大予測言語数の設定


In [6]:
parameter_dict = {}
train_path = "oda201512handson/train/"
test_path = "oda201512handson/test/"
#--------Hands on  2----------------------------------------------------------------
# #

mode = "train"
parameter_dict["source"] = train_path + "train1000.ja"
parameter_dict["target"] = train_path + "train1000.en"
parameter_dict["test_source"] = test_path + "test1000.ja"
parameter_dict["test_target"] = test_path + "test1000_hyp.en"
parameter_dict["reference_target"] = test_path + "test1000.en"
parameter_dict["vocab"] = 550
parameter_dict["embed"] = 500
parameter_dict["hidden"] = 20
parameter_dict["epoch"] = 20
parameter_dict["minibatch"] = 64
parameter_dict["generation_limit"] = 256
parameter_dict["show_hands_on_number"] = 10
parameter_dict["show_i_epoch"] = 0
#--------Hands on  2----------------------------------------------------------------#

## 5.実行

In [7]:
trace('initializing ...')
wrapper.init()

encoderDecoderModel = EncoderDecoderModelForward(parameter_dict)
encoderDecoderModel.train_model()

2015-12-07 17:55:40.853534 ... initializing ...
2015-12-07 17:55:40.854262 ... making vocaburaries ...
2015-12-07 17:55:40.864812 ... making model ...
2015-12-07 17:55:40.892242 ... epoch 1/20: 
2015-12-07 17:55:41.029790 ... epoch   1/ 20, sample        1
2015-12-07 17:55:41.030170 ...   src = エミ は 幸せ そう に 見え ま す 。 * * * * *
2015-12-07 17:55:41.030415 ...   trg = emi looks happy . * *
2015-12-07 17:55:41.030678 ...   hyp = why who have <unk> much old
2015-12-07 17:55:43.023232 ... saving model ...
2015-12-07 17:55:43.568263 ... epoch 2/20: 
2015-12-07 17:55:43.676158 ... epoch   2/ 20, sample        1
2015-12-07 17:55:43.676536 ...   src = エミ は 幸せ そう に 見え ま す 。 * * * * *
2015-12-07 17:55:43.676862 ...   trg = emi looks happy . * *
2015-12-07 17:55:43.677117 ...   hyp = i is <unk> is * *
2015-12-07 17:55:45.827144 ... saving model ...
2015-12-07 17:55:46.336370 ... epoch 3/20: 
2015-12-07 17:55:46.443839 ... epoch   3/ 20, sample        1
2015-12-07 17:55:46.444218 ...   src = エミ は 幸せ 

## 6.学習したモデルを使用したテスト (Advanced)

学習したモデルを使用してテスト

* 学習したモデルを利用してテストデータ（日本語）を英語に翻訳しモデルに保存。

In [8]:
mode = "test"
model_name = "ChainerMachineTranslation.021"
trace('initializing ...')
wrapper.init()

encoderDecoderModel = EncoderDecoderModelForward(parameter_dict)
encoderDecoderModel.test_model(model_name)

2015-12-07 17:58:39.214421 ... initializing ...
2015-12-07 17:58:39.216559 ... loading model ...
2015-12-07 17:58:39.527824 ... generating translation ...
2015-12-07 17:58:39.543324 ... sample        1 -       64 ...
2015-12-07 17:58:39.605777 ... sample       65 -      128 ...


it 's my father is out .
he was <unk> on the <unk> .
't be <unk> .
she is a <unk> <unk> to the <unk> .
i don 't be good <unk> in the hospital .
i <unk> was he was to me to me a <unk> .
i have been to <unk> to the <unk> to the <unk> .
it is no no <unk> is this <unk> than this afternoon .
i am <unk> <unk> to you to do it .
this are <unk> for <unk> .
my has no been mother of his <unk> .
i 'm a <unk> on the <unk> .
i want to do to the <unk> to the <unk> .
no no friend is to old .
i don 't have been to <unk> to the <unk> .
she was <unk> in the <unk> .
it is been been been <unk> to <unk> a <unk> of the town .
i 'm a <unk> to the <unk> to the <unk> .
we is the <unk> to the <unk> to the <unk> .
my which always <unk> <unk> to the <unk> to the <unk> .
i have the <unk> as <unk> to the <unk> .
he was he was kind to do .
i was <unk> to you to do to me ?
i was <unk> was he was <unk> .
i have the <unk> , i 'll be <unk> to the <unk> .
the japan <unk> <unk> is the <unk> than he is the <unk> .
i <unk> <

2015-12-07 17:58:39.666370 ... sample      129 -      192 ...



they are <unk> to his <unk> .
what is it to do it to do it .
it is no no <unk> is to old .
i don 't have <unk> to <unk> to the <unk> .
my <unk> <unk> to <unk> a <unk> of his <unk> .
i have been <unk> to the <unk> .
what did you <unk> for you <unk> for the <unk> .
i 'll you go to be late for school .
i <unk> you <unk> to the <unk> .
you is <unk> is the <unk> to the <unk> to the <unk> .
i <unk> for this <unk> to the <unk> .
you are you <unk> to you to do it .
i have to do to be <unk> .
he has been out in the <unk> .
you are you going to do you to do it ?
i not you going to the <unk> .
it is a do not take the <unk> to the <unk> .
she is in the <unk> is to take <unk> to the <unk> .
he is <unk> to the <unk> to the <unk> .
<unk> is <unk> 't <unk> to the <unk> .
i don 't have been to <unk> to the <unk> .
he is <unk> a <unk> of his <unk> of his <unk> .
i have a <unk> to the <unk> to the <unk> .
i have the time to do <unk> to the <unk> .
the time as i <unk> to see the <unk> .
i 'll he <unk> 't

2015-12-07 17:58:39.735267 ... sample      193 -      256 ...


...


i 'll never <unk> of his work .
he is to take <unk> to the <unk> .
he is a <unk> to take the <unk> to the <unk> .
i 'm a <unk> to the <unk> to the <unk> .
they are <unk> he was <unk> .
i have a <unk> to have been to <unk> .
no i 'm he will .
he was <unk> on the <unk> .
i <unk> you <unk> to <unk> to the <unk> .
i am to do it .
he is the <unk> to his <unk> on the <unk> .
i <unk> a <unk> to the <unk> to the <unk> .
he can 't come to do .
i <unk> <unk> to <unk> to the <unk> .
my <unk> is the <unk> is <unk> to the <unk> .
how is the <unk> <unk> to the <unk> to the <unk> .
i will be ready soon .
we have you have been to do it ?
i am .
i don 't have been to <unk> to the <unk> .
there 's a problem <unk> to be <unk> .
i have the <unk> is the <unk> of yourself .
it is no <unk> of his <unk> .
could you call me what i am , you have to do it .
i have the time to do to do it ?
i have the <unk> , but the <unk> <unk> to the <unk> .
don be kind be good in japan ?
i don 't have <unk> to <unk> to the <u

2015-12-07 17:58:40.355255 ... sample      897 -      960 ...



she is more <unk> to have been to <unk> .
he is the <unk> as as as as i am .
i have been to do to do you he 's <unk> .
i 'll you do you have to do it ?
he <unk> on the <unk> .
i <unk> as <unk> to you to go to the <unk> .
i <unk> my <unk> 't <unk> the <unk> .
this book is <unk> of to that <unk> in the <unk> .
he had better his mother of <unk> .
he was <unk> on the <unk> .
do you <unk> , i not it to do it .
they he is the <unk> to be <unk> .
you can you are you ?
can will you going to the <unk> to the <unk> .
i was <unk> as i <unk> to the <unk> .
she is a <unk> than this <unk> .
i was <unk> to <unk> to the <unk> .
i have a <unk> to the <unk> to the <unk> .
he is not come to go to the <unk> .
he is a good <unk> for the <unk> .
i want to do to do it .
i 'm a <unk> 't <unk> a <unk> .
my <unk> is not my mother <unk> to the <unk> .
how is no <unk> than this <unk> .
my brother about to <unk> .
my <unk> are you <unk> to the <unk> .
i <unk> <unk> to have to <unk> to the <unk> .
i have any any m

2015-12-07 17:58:40.412203 ... sample      961 -     1000 ...
2015-12-07 17:58:40.458256 ... finished.



can this can 't do you <unk> .
this <unk> is <unk> , but the <unk> <unk> to the <unk> .
i have not have been to be <unk> .
i have been <unk> to the <unk> .
my <unk> is the <unk> is the <unk> of his <unk> .
his work is not come to go to the <unk> .
i don 't to do you to do it ?
it is no mother of his <unk> .
he is <unk> to the <unk> .
any rain <unk> as i have been to <unk> .
this <unk> is <unk> , but <unk> to the <unk> .
i 'll what what i 'll you <unk> to the <unk> .
i 'll 't have been a <unk> .
you do <unk> to do to do it ?
<unk> is <unk> is <unk> , but he <unk> of the <unk> .
it you be <unk> to be <unk> .
he is not to have been to go to the <unk> .
he is always been a <unk> of the <unk> .
he <unk> <unk> to the <unk> on the <unk> .
the child is there 's in the <unk> is more <unk> .
he <unk> <unk> to the <unk> to the <unk> .
he was <unk> to the <unk> to the <unk> .
go is the <unk> to the <unk> .
could i want to do to do to you to do ?
she did you <unk> to <unk> to the <unk> .
i have a 

## 7.学習したモデルを評価 (Advanced)

学習したモデルの評価するため、BLEUを算出

* BlEUとは翻訳の客観的評価に使用される指標で、答えとなる文章との一致率を評価する方法を用いています。
詳しく知りたい方は下記をご覧ください。
http://www2.nict.go.jp/univ-com/multi_trans/member/mutiyama/corpmt/4.pdf

In [14]:
cmd_corpus = "mteval-corpus  -e BLEU RIBES -r " +parameter_dict["reference_target"] + " -h " + parameter_dict["test_target"]
cmd_sentence = "mteval-sentence  -e BLEU RIBES -r " +parameter_dict["reference_target"] + " -h " + parameter_dict["test_target"]
mteval_corpus = subprocess.Popen(cmd_corpus, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout_data = mteval_corpus.stdout.read()
print(stdout_data.decode('utf-8'))

BLEU=0.000000	RIBES=0.417212

